In [1]:
from config import *
from data_processer import CSCDataset
from torch.utils.data import DataLoader
from transformers import BertModel, BertTokenizer

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import (
    AdamW,
    BertForSequenceClassification,
    BertTokenizer,
    get_scheduler,
)

In [3]:
# matrices
def cal_err(raw_sentence, pred_sentence, corr_sentence):
    matrices = ["over_corr", "total_err", "true_corr"]

    char_level = {key: 0 for key in matrices}
    sent_level = {key: 0 for key in matrices}

    for i, c in enumerate(raw_sentence):
        pc, cc = pred_sentence[i], corr_sentence[i]
        f1 = f2 = False

        if cc != c:
            char_level["total_err"] += 1
            char_level["true_corr"] += pc == cc
            f1 = True
        elif pc != cc:
            char_level["over_corr"] += 1
            f2 = True

    # true_corr 未计算
    sent_level["total_err"] += f1
    sent_level["over_corr"] += f2

    return char_level, sent_level

In [4]:
def cal_err(raw_sentence, pred_sentence, corr_sentence):
    matrices = ["over_corr", "total_err", "true_corr"]
    char_level = {key: 0 for key in matrices}
    sent_level = {key: 0 for key in matrices}

    for i, c in enumerate(raw_sentence):
        pc, cc = pred_sentence[i], corr_sentence[i]
        f1 = f2 = 0

        if cc != c:
            char_level["total_err"] += 1
            char_level["true_corr"] += pc == cc
            f1 = 1
        elif pc != cc:
            char_level["over_corr"] += 1
            f2 = 1

    # true_corr 未计算
    sent_level["total_err"] += f1
    sent_level["over_corr"] += f2

    return char_level, sent_level

In [5]:
def test(model, tokenizer, test_data_loader):
    model.eval()
    total_loss = 0
    matrices = ["over_corr", "total_err", "true_corr"]
    test_char_level = {key: 0 for key in matrices}
    test_sent_level = {key: 0 for key in matrices}

    with torch.no_grad():
        for batch in test_data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device).type(torch.float)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, src_mask=attention_mask)
            logits = outputs.permute(0, 2, 1)

            loss = cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
            total_loss += loss.item()

            t = torch.argmax(outputs, dim=-1)
            nt = t * attention_mask
            pred = tokenizer.batch_decode(nt, skip_special_tokens=True)

            for i in range(len(t)):
                char_level, sent_level = cal_err(input_ids[i], nt[i], labels[i])
                test_char_level = {
                    key: test_char_level[key] + v for key, v in char_level.items()
                }
                test_sent_level = {
                    key: test_sent_level[key] + v for key, v in sent_level.items()
                }
        print(total_loss / len(test_data_loader), test_char_level, test_sent_level)

In [6]:
from config import *
from models import Seq2SeqModel
from torch.nn.functional import cross_entropy
from tqdm import tqdm

# temp_data
nhead = 4  # 多头注意力机制的头数
num_decoder_layers = 2
dim_feedforward = 3072
max_seq_len = 128
dropout = 0.1

if __name__ == "__main__":
    tokenizer = BertTokenizer.from_pretrained(checkpoint)
    bert = BertModel.from_pretrained(checkpoint)

    train_dataset = CSCDataset([SIGHAN_train_dir_err, SIGHAN_train_dir_corr], tokenizer)
    train_data_loader = DataLoader(
        train_dataset, num_workers=0, shuffle=True, batch_size=8
    )

    test_dataset = CSCDataset(
        [SIGHAN_train_dir_err14, SIGHAN_train_dir_corr14], tokenizer
    )
    test_data_loader = DataLoader(
        test_dataset, num_workers=0, shuffle=True, batch_size=4
    )

    model = Seq2SeqModel(
        bert, nhead, num_decoder_layers, dim_feedforward, max_seq_len, dropout
    )

    optimizer = AdamW(model.parameters(), lr=learning_rate)

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    epochs = 10
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(
            enumerate(train_data_loader),
            desc=f"Epoch:{epoch+1}/{epochs}",
            total=len(train_data_loader),
        )

        for i, batch in progress_bar:
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device).type(torch.float)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, src_mask=attention_mask)
            logits = outputs.permute(0, 2, 1)  # (batch_size, vocab_size, seq_len)

            # 反向传播在这，故labels不需要传入模型
            loss = cross_entropy(logits, labels, ignore_index=tokenizer.pad_token_id)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            progress_bar.set_postfix({"loss": "{:.3f}".format(loss.item())})

        t = torch.argmax(outputs, dim=-1)
        nt = t * attention_mask
        pred = tokenizer.batch_decode(nt, skip_special_tokens=True)
        # print(pred)
        # print(f"origin{tokenizer.batch_decode(labels, skip_special_tokens=True)}")

        for i, v in enumerate(nt):
            r, l = input_ids[i], labels[i]
            print(tokenizer.decode(r, skip_special_tokens=True))
            print(tokenizer.decode(v, skip_special_tokens=True))
            print(tokenizer.decode(l, skip_special_tokens=True))
            print(cal_err(r, v, l))

        print(f"Epoch {epoch+1} Loss: {total_loss / len(train_data_loader)}")
        test(model, tokenizer, test_data_loader)

preprocessing sighan dataset: 2339it [00:00, 77618.22it/s]
preprocessing sighan dataset: 100%|███████████████████████████████████████████| 2339/2339 [00:00<00:00, 1172660.42it/s]


共2339句，共73264字，最长的句子有171字


preprocessing sighan dataset: 3437it [00:00, 686467.75it/s]
preprocessing sighan dataset: 100%|███████████████████████████████████████████| 3437/3437 [00:00<00:00, 1145112.63it/s]


共3437句，共170330字，最长的句子有258字


E:\Anaconda3\envs\csc_env\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch:1/10: 100%|████████████████████████████████████████████████████████| 585/585 [01:12<00:00,  8.12it/s, loss=0.588]


...

有 一 天 他 問 我 朋 友 的 手 機 號 碼 ， 我 的 朋 友 可 以 告 訴 他 號 嗎 因 為 相 信 他 。
有 一 天 他 問 我 朋 友 的 手 機 號 碼 ， 我 的 朋 友 可 以 告 訴 他 號 媽 因 為 相 信 他 。
有 一 天 他 問 我 朋 友 的 手 機 號 碼 ， 我 的 朋 友 可 以 告 訴 他 號 碼 因 為 相 信 他 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 0, 'true_corr': 0})
他 看 到 了 一 輛 紅 汽 車 ， 就 決 定 叫 那 個 人 定 車 。
他 看 到 了 一 輛 紅 汽 車 ， 就 決 定 叫 那 個 人 定 車 。
他 看 到 了 一 輛 紅 汽 車 ， 就 決 定 叫 那 個 人 停 車 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 0, 'true_corr': 0})
由 於 年 紀 已 經 不 輕 ， 所 以 要 再 從 新 找 一 份 工 作 ， 對 他 來 說 是 如 此 艱 難 的 事 啊 ！
由 於 年 紀 已 經 不 輕 ， 所 以 要 再 從 新 找 一 份 工 作 ， 對 他 來 說 是 如 此 艱 難 的 事 啊 ！
由 於 年 紀 已 經 不 輕 ， 所 以 要 再 重 新 找 一 份 工 作 ， 對 他 來 說 是 如 此 艱 難 的 事 啊 ！
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 0, 'true_corr': 0})
Epoch 8 Loss: 0.18372087489463326
0.37691865138016467 {'over_corr': 2528, 'total_err': 5278, 'true_corr': tensor

Epoch:9/10: 100%|████████████████████████████████████████████████████████| 585/585 [01:37<00:00,  6.02it/s, loss=0.212]


你 的 公 交 孩 子 真 實 是 普 通 的 事 況 ， 但 是 我 自 己 又 已 經 有 了 生 計 畫 ， 所 以 現 在 沒 辦 法 去 做 控 制 你 的 公 司 。
你 的 公 交 孩 子 真 實 是 普 通 的 事 況 ， 但 是 我 自 己 又 已 經 有 了 生 計 畫 ， 所 以 現 在 沒 辦 法 去 做 控 制 你 的 公 司 。
你 的 公 交 孩 子 真 實 是 普 通 的 事 況 ， 但 是 我 自 己 又 已 經 有 人 生 計 畫 ， 所 以 現 在 沒 辦 法 去 做 控 制 你 的 公 司 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 0, 'true_corr': 0})
今 天 大 明 第 一 次 到 十 大 去 ， 他 是 新 生 在 師 大 。
今 天 大 明 第 一 次 到 十 大 去 ， 他 是 新 生 在 師 大 。
今 天 大 明 第 一 次 到 師 大 去 ， 他 是 新 生 在 師 大 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 0, 'true_corr': 0})
她 教 我 怎 嗎 去 學 校 了 。
她 教 我 怎 麼 去 學 校 了 。
她 教 我 怎 麼 去 學 校 了 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(1, device='cuda:0')}, {'over_corr': 0, 'total_err': 0, 'true_corr': 0})
Epoch 9 Loss: 0.15384562173460284
0.37297898604599544 {'over_corr': 2578, 'total_err': 5278, 'true_corr': tensor(492, device='cuda:0')} {'over_corr': 0, 'total_err': 0, 'tr

Epoch:10/10: 100%|███████████████████████████████████████████████████████| 585/585 [01:42<00:00,  5.68it/s, loss=0.192]


這 個 旅 行 是 天 兩 夜 ， 印 度 的 南 部 很 漂 亮 ， 風 景 也 很 美 。
這 個 旅 行 是 天 兩 夜 ， 印 度 的 南 部 很 漂 亮 ， 風 景 也 很 美 。
這 個 旅 行 四 天 兩 夜 ， 印 度 的 南 部 很 漂 亮 ， 風 景 也 很 美 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 0, 'true_corr': 0})
他 從 半 年 前 起 ， 一 直 在 醫 院 裡 ， 可 能 她 將 來 會 沒 希 望 了 。 如 果 他 死 掉 的 ， 你 能 負 責 任 嗎 ？
他 從 半 年 前 起 ， 一 直 在 醫 院 裡 ， 可 能 她 將 來 會 沒 希 望 了 。 如 果 他 死 掉 的 ， 你 能 負 責 任 嗎 ？
他 從 半 年 前 起 ， 一 直 在 醫 院 裡 ， 可 能 她 將 來 會 沒 希 望 了 。 如 果 他 死 掉 了 ， 你 能 負 責 任 嗎 ？
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 0, 'true_corr': 0})
醫 生 跟 他 說 ： 你 一 定 最 少 每 個 星 期 運 動 兩 次 ， 要 不 然 你 的 身 體 不 輸 服 ， 你 的 頭 會 容 易 疼 。
醫 生 跟 他 說 ： 你 一 定 最 少 每 個 星 期 運 動 兩 次 ， 要 不 然 你 的 身 體 不 輸 服 ， 你 的 頭 會 容 易 疼 。
醫 生 跟 他 說 ： 你 一 定 最 少 每 個 星 期 運 動 兩 次 ， 要 不 然 你 的 身 體 不 舒 服 ， 你 的 頭 會 容 易 疼 。
({'over_corr': 0, 'total_err': 1, 'true_corr': tensor(0, device='cuda:0')}, {'over_corr': 0, 'total_err': 0, 'true_corr': 0})
Epoch 10 L

In [7]:
# 测试函数
def test_model(model, tokenizer, text, max_length=128):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Start with the special token [CLS]
        start_token = tokenizer.cls_token_id
        decoded_indices = [start_token]

        for _ in range(max_length - 1):
            tgt_tensor = torch.tensor(decoded_indices).unsqueeze(1).to(device)
            outputs = model(input_ids, tgt=tgt_tensor, src_mask=attention_mask)
            next_token_logits = outputs[-1, :, :].squeeze()
            next_token_id = torch.argmax(next_token_logits).item()

            if next_token_id == tokenizer.sep_token_id:
                break

            decoded_indices.append(next_token_id)

        corrected_text = tokenizer.decode(decoded_indices, skip_special_tokens=True)
        return corrected_text


# 测试
test_sentence = "这是个测试句子。"
print("Original Sentence:", test_sentence)
print("Corrected Sentence:", test_model(model, tokenizer, test_sentence))

Original Sentence: 这是个测试句子。


TypeError: Seq2SeqModel.forward() got an unexpected keyword argument 'tgt'

In [ ]:
class Trainer:
    def __init__(self, model):
        self.model = model
        self.matrices = ["over_corr", "total_err", "true_corr"]

    def train(self, dataloader, epoch):
        self.iteration(dataloader, epoch)

    def test(self, dataloader):
        self.iteration(dataloader, train=False)

    def iteration(self, dataloader, epochs=1, train=True):
        mode = "train" if train else "dev"
        model.train() if train else model.eval()

        for epoch in range(epochs):
            # matrices
            total_loss = 0
            char_level = {key: 0 for key in self.matrices}
            sent_level = {key: 0 for key in self.matrices}

            progress_bar = tqdm(
                dataloader,
                desc=f"{mode} Epoch:{epoch+1}/{epochs}",
                total=len(dataloader),
            )
            for batch in progress_bar:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids, src_mask=attention_mask, tgt=labels)
                logits = outputs.permute(0, 2, 1)  # (batch_size, vocab_size, seq_len)

                loss = cross_entropy(
                    logits, labels, ignore_index=tokenizer.pad_token_id
                )

                total_loss += loss.item()
                progress_bar.set_postfix({"loss": "{:.3f}".format(loss.item())})

    def cal_err(raw_sentence, pred_sentence, corr_sentence):
        char_level = {key: 0 for key in self.matrices}
        sent_level = {key: 0 for key in self.matrices}

        for i, c in enumerate(raw_sentence):
            pc, cc = pred_sentence[i], corr_sentence[i]
            f1 = f2 = False

            if cc != c:
                char_level["total_err"] += 1
                char_level["true_corr"] += pc == cc
                f1 = True
            elif pc != cc:
                char_level["over_corr"] += 1
                f2 = True

        # true_corr 未计算
        sent_level["total_err"] += f1
        sent_level["over_corr"] += f2

        return char_level, sent_level